<a href="https://colab.research.google.com/github/Safarbe/Scoring/blob/main/Correlated_ensembles_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score, auc 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier      
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from catboost import CatBoostClassifier
import lightgbm as lgb
import numpy as np
#from imblearn.over_sampling import RandomOverSampler
import  matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.2 MB/s eta 0:00:00


In [4]:
train = pd.read_csv('/content/Train.csv')
test = pd.read_csv('/content/Test.csv')

In [10]:
dataset = pd.concat([train, test]).reset_index()

In [11]:
dtype_objs = ['product_type', 'loan_purpose', 'filial', 'gender', 'marital_status', 'client_type', 'income_type', 'has_car', 'has_house', 'guarantor', 'zalog']

In [12]:
corr = dataset.drop(['is_bad_30'], axis = 1).corrwith(train['is_bad_30']).sort_values(ascending=False)

In [26]:
train_corr = pd.DataFrame({'features':corr.index, 'corr':corr.values})

In [14]:
le = LabelEncoder()
for i in range(len(dtype_objs)):
  dataset[dtype_objs[i-1]] = le.fit_transform(dataset[dtype_objs[i-1]])

In [18]:
dataset = dataset.drop(['index'], axis = 1)

In [19]:
X, y = dataset.drop(['is_bad_30'], axis = 1), dataset['is_bad_30']

In [32]:
X_new = pd.concat([X[dtype_objs], X[list(train_corr['features'][:15])]], axis=1, sort=False)
y_new = y

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [33]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size = 0.25)

In [43]:
model_rfc = RandomForestClassifier()
model_rfc.fit(X_train, y_train)

model_rfc_new = RandomForestClassifier()
model_rfc_new.fit(X_train_new, y_train_new)


model_gbc = GradientBoostingClassifier()
model_gbc.fit(X_train, y_train)

model_gbc_new = GradientBoostingClassifier()
model_gbc_new.fit(X_train_new, y_train_new)



model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

model_lgb_new = lgb.LGBMClassifier()
model_lgb_new.fit(X_train_new, y_train_new)



model_cbct = CatBoostClassifier()
model_cbct.fit(X_train, y_train)

model_cbct_new = CatBoostClassifier()
model_cbct_new.fit(X_train_new, y_train_new)

Learning rate set to 0.024196
0:	learn: 0.6557825	total: 55.4ms	remaining: 55.4s
1:	learn: 0.6211511	total: 62ms	remaining: 30.9s
2:	learn: 0.5851497	total: 73.6ms	remaining: 24.4s
3:	learn: 0.5561692	total: 82.2ms	remaining: 20.5s
4:	learn: 0.5295416	total: 88.7ms	remaining: 17.7s
5:	learn: 0.5045249	total: 95.4ms	remaining: 15.8s
6:	learn: 0.4803808	total: 102ms	remaining: 14.5s
7:	learn: 0.4586511	total: 109ms	remaining: 13.5s
8:	learn: 0.4377692	total: 116ms	remaining: 12.7s
9:	learn: 0.4188356	total: 123ms	remaining: 12.2s
10:	learn: 0.3993008	total: 131ms	remaining: 11.8s
11:	learn: 0.3837092	total: 138ms	remaining: 11.3s
12:	learn: 0.3691110	total: 144ms	remaining: 10.9s
13:	learn: 0.3553043	total: 151ms	remaining: 10.6s
14:	learn: 0.3427923	total: 157ms	remaining: 10.3s
15:	learn: 0.3312719	total: 164ms	remaining: 10.1s
16:	learn: 0.3203317	total: 171ms	remaining: 9.91s
17:	learn: 0.3094664	total: 178ms	remaining: 9.74s
18:	learn: 0.2974876	total: 186ms	remaining: 9.59s
19:	lea

In [40]:
def thresh_test(author, model_name, model, X_test, y_test):
    threshholds = np.arange(0.25, 0.09, -0.01)
    matrix = []
    for i in threshholds:
        y_pred = np.where(model.predict_proba(X_test)[:, 1] > i, 1, 0)
        conf_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])
        matrix.append([author, model_name, i, roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]), 
                    accuracy_score(y_test, y_pred), conf_matrix[1][1], conf_matrix[0][1], conf_matrix[0][0], conf_matrix[1][0]])
    output = pd.DataFrame(matrix, columns=['Author', 'Model', 'Threshold', 'ROC_AUC', 'Accuracy', 'TN', 'FP', 'TP', 'FN'])
    return output

In [48]:
model_names = ['RandomForestClassifier', 'RandomForestClassifier_corr', 'GradientBoostingClassifier', 'GradientBoostingClassifier_corr',
               'LGBMClassifier', 'LGBMClassifier_corr', 'CatBoostClassifier', 'CatBoostClassifier_corr'
               ]

models = [model_rfc, model_rfc_new, model_gbc, model_gbc_new, model_lgb, model_lgb_new, model_cbct, model_cbct_new]


X_tests = [X_test, X_test_new, X_test, X_test_new, X_test, X_test_new, X_test, X_test_new]
y_tests = [y_test, y_test_new, y_test, y_test_new, y_test, y_test_new, y_test, y_test_new]

In [47]:
model_names[7]

'CatBoostClassifier_corr'

In [50]:
for i in range(len(model_names)):
  print(thresh_test('Safarbek', model_names[i-1], models[i-1], X_tests[i-1], y_tests[i-1]))

      Author                    Model  Threshold   ROC_AUC  Accuracy  TN   FP  \
0   Safarbek  CatBoostClassifier_corr       0.25  0.861708  0.962647  12   14   
1   Safarbek  CatBoostClassifier_corr       0.24  0.861708  0.962241  14   17   
2   Safarbek  CatBoostClassifier_corr       0.23  0.861708  0.961835  14   18   
3   Safarbek  CatBoostClassifier_corr       0.22  0.861708  0.962241  15   18   
4   Safarbek  CatBoostClassifier_corr       0.21  0.861708  0.963053  17   18   
5   Safarbek  CatBoostClassifier_corr       0.20  0.861708  0.963459  20   20   
6   Safarbek  CatBoostClassifier_corr       0.19  0.861708  0.962647  24   26   
7   Safarbek  CatBoostClassifier_corr       0.18  0.861708  0.962241  28   31   
8   Safarbek  CatBoostClassifier_corr       0.17  0.861708  0.961429  31   36   
9   Safarbek  CatBoostClassifier_corr       0.16  0.861708  0.961429  34   39   
10  Safarbek  CatBoostClassifier_corr       0.15  0.861708  0.956963  34   50   
11  Safarbek  CatBoostClassi